In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os

tf.keras.backend.set_floatx('float32')

#Number of grid points along x and y axis
N = 101

#domain size 
min = -np.pi
max = np.pi

h = (max - min)/(N-1)
e = 1.5*h

#penalty constant
M1 = 100
A0 = 5.0

#weight 
omega = 0.1

In [ ]:
m = 100
n = 100

h = 2*np.pi/m
e = 1.5*h
M1 = 100
M2 = 0

w = 0.1

alpha = 5.00
beta = 4.6

In [ ]:
#This function considers all the inner points and organize the nodal values for the calculation of e
def organize_interaction(A):

    first = tf.concat([tf.reshape(A[1:, 1:-1], (-1, 1)), tf.reshape(A[0:-1, 1:-1], (-1, 1))], axis=1)
    
    second = tf.concat([tf.reshape(A[1:-1, 1:], (-1, 1)), tf.reshape(A[1:-1, 0:-1], (-1, 1))], axis=1)

    input = tf.concat([first, second], axis=0)
    
    return input

In [ ]:
def calc_PM(Nx, Ny):

    #number of divisions along x and y
    m = Nx - 1
    n = Ny - 1

    num_interactions = (n-1)*m + (m-1)*n
    num_inner_nodes = (m-1)*(n-1)
    
    #first gap --> number of zeros in between -1 or 1. Note that we start with vertical interactions and then horizontal interactions
    first_gap = n-2
    second_gap = (n-2) + (n-1)*(m-2)
    
    indices = np.zeros((4*num_inner_nodes, 2))
    values = np.zeros((4*num_inner_nodes, ))
    dense_size = np.array([num_inner_nodes, num_interactions]) 
    
    current = 0
    for i in range(num_inner_nodes):
        indices[current][:] = np.array([i, i])
        values[current] = 1.
        current = current + 1
        
        indices[current][:] = np.array([i, i+1+first_gap])
        values[current] = -1.
        current = current + 1

        indices[current][:] = np.array([i, i+1+first_gap+1+second_gap + int(i/(n-1))])
        values[current] = 1.
        current = current + 1

        indices[current][:] = np.array([i, i+1+first_gap+1+second_gap+1 + int(i/(n-1))])
        values[current] = -1.
        current = current + 1

    values = tf.convert_to_tensor(values, dtype='float32')
    
    PM = tf.sparse.SparseTensor(indices, values, dense_size)
    
    return PM

In [ ]:
def create_model(input_shape, nhu=2, npl=32):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Input(input_shape))

    for _ in range(nhu):
        model.add(tf.keras.layers.Dense(npl,
                                        activation='relu',
                                        kernel_initializer='glorot_normal'))
    
    model.add(tf.keras.layers.Dense(1, activation='tanh'))

    return model

In [ ]:
#Reshapes H into (Nx - 2)*(Ny - 2) matrix and add 0s for the boundary
def reshape_H(Nx, Ny, A):
    A = tf.reshape(A, (Nx-2, Ny-2))
    top_and_bottom = tf.zeros((Nx-2, 1), dtype='float32')

    A = tf.concat([top_and_bottom, A, top_and_bottom], axis=1)

    left_and_right = tf.zeros((1, Ny), dtype='float32')

    A = tf.concat([left_and_right, A, left_and_right], axis=0)

    return A

In [ ]:
#takes the values of inner grid points and add boundary values
def update_boundary(Nx, Ny, H):

    #adding boundary points
    top_n_bottom = -1.*tf.ones((Nx-2, 1), dtype='float32')
    
    output = tf.concat([top_n_bottom, H, top_n_bottom], axis=1)
    
    right_n_left = -1.*tf.ones((1, Ny), dtype='float32')

    output = tf.concat([right_n_left, output, right_n_left], axis=0)

    return output


In [ ]:
#Elastic Bending Energy with area constraint
def compute_energy(phi):
    laplace = (phi[1:-1, 2:] + phi[1:-1, 0:-2] + phi[2:, 1:-1] + phi[0:-2, 1:-1]
            - 4*phi[1:-1, 1:-1])/(h*h)
            
    I = 0.5*e*tf.math.square(laplace - (tf.math.square(phi[1:-1, 1:-1]) - 1)*phi[1:-1, 1:-1]/(e*e))

    #riemann sum
    first_integral = h*h*tf.reduce_sum(I)
    area = 0.5*h*h*tf.reduce_sum(phi[1:-1, 1:-1] + 1.)
    
    energy = first_integral + M1*(area - A0)**2 
    
    return energy, area


In [ ]:
#Algorithm 2 from the paper
#M is the number of message passing steps
#fR and fO are the networks
def march_forward(M, phi_in, fR, fO, PM):

    Nx = phi_in.shape[0]
    Ny = phi_in.shape[1]
    
    H = tf.zeros(phi_in.shape, dtype='float32')
    
    for _ in range(M):
        input_phi = organize_interaction(phi_in)
        input_H = organize_interaction(H)
        input1 = tf.concat([input_phi, input_H], axis=1)

        
        output1 = fR(input1)

        c = tf.sparse.sparse_dense_matmul(PM, output1)
        
        input2 = tf.concat((tf.reshape(phi_in[1:-1, 1:-1], (-1, 1)), c), axis=1)
        
        output2 = fO(input2)
        
        H = reshape_H(Nx, Ny, output2)
    
    #adding the boundary values
    phi_out = update_boundary(Nx, Ny, H[1:-1, 1:-1])
    return phi_out

In [ ]:
def compute_loss(M, phi_in, fR, fO, PM, initial=False):
    phi_out = march_forward(M, phi_in, fR, fO, PM)
    
    energy_in, area_in = compute_energy(phi_in)
    energy_out, area_out = compute_energy(phi_out)
    
    if initial:
        loss = tf.reduce_mean(tf.math.square(phi_in - phi_out))
    else:
        loss = tf.math.reduce_mean(tf.math.square(phi_out - phi_in)) + omega*energy_out/energy_in
    
    return energy_in, energy_out, loss, area_out, area_in

In [ ]:
#initialize phi
def initialize_phi(Nx, Ny):
    phi = -1.*np.ones((Nx, Ny))
    

    for i in range(Nx):
        for j in range(Ny):
            x = min + i*h
            y = min + j*h

            #ellipse
            a = 0.5*np.pi
            b = 0.25*np.pi
            phi[i,j] = np.tanh((1.0 - (x/a)**2 - (y/b)**2)/(np.sqrt(2)*e))
            
    return tf.convert_to_tensor(phi, dtype='float32')

In [ ]:
class FONN():
    def __init__(self, M):
        self.M = M
        self.PM = calc_PM(N, N)
        
        self.fR = create_model(input_shape=(4, ))
        self.fO = create_model(input_shape=(2, ))
        
        self.phi_0 = initialize_phi(N, N)
        self.phi = initialize_phi(N, N)
    
    def initial_training(self, epochs):
        optimizer = tf.keras.optimizers.Adam()

        for ep in range(epochs):

            noise = tf.random.normal(self.phi_0.shape, mean=0.0, stddev=0.05, dtype='float32')
            noisy_input = self.phi_0 + noise

            with tf.GradientTape() as tape:
                energy_in, energy_out, loss, area_out, area_in = compute_loss(self.M, noisy_input, self.fR, self.fO, self.PM, initial=True)
            grads = tape.gradient(loss, tape.watched_variables())
            
            optimizer.apply_gradients(zip(grads, tape.watched_variables()))
            del tape

            print("epoch = ", ep, " loss = ", loss.numpy())
    
    #K -> fine tuning steps
    def progressive_method(self, steps, K, save_plots=False, location=""):
        optimizer = tf.keras.optimizers.Adam()
        
        #open the file
        if save_plots:
            name_E = location + "/energy.txt"
            name_A = location + "/A.txt"
            file_E = open(name_E, "w")
            file_A = open(name_A, "w")

        for step in range(steps):
            
            for ep in range(K):
                with tf.GradientTape() as tape:
                    energy_in, energy_out, loss, area_out, area_in = compute_loss(self.M, self.phi, self.fR, self.fO, self.PM)
                
                grads = tape.gradient(loss, tape.watched_variables())

                optimizer.apply_gradients(zip(grads, tape.watched_variables()))
                del tape

            self.phi = march_forward(self.M, self.phi, self.fR, self.fO, self.PM)

            print("step = ", step, " energy_in = ", energy_in.numpy(), " energy_out = ", energy_out.numpy(), " area_in ", area_in.numpy(), " area_out ", area_out.numpy())
            
            #if energy gets 0 
            if np.abs(energy_out.numpy()) < 10e-5:
                data_file_name = location + "/dat_" + str(step) + ".txt"
                np.savetxt(data_file_name, np.reshape(self.phi.numpy(), -1))
                fig_name = location + "/" + str(step) + ".pdf"
                title_ = "$E(phi)$ = " + str(compute_energy(self.phi)[0].numpy())
                plt.figure()
                plt.contourf(tf.transpose(self.phi))
                plt.colorbar()
                plt.title(title_)
                plt.savefig(fig_name)
                plt.close()
                break
            
            #To save plots and result
            if save_plots:
                result_E = str(energy_in.numpy()) + "\t" + str(energy_out.numpy()) + "\n"
                result_A = str(area_in.numpy()) + "\t" + str(area_out.numpy()) + "\n"
                file_E.write(result_E)
                file_A.write(result_A)
                
                #save figure
                if step % 25 == 0:
                    data_file_name = location + "/dat_" + str(step) + ".txt"
                    np.savetxt(data_file_name, np.reshape(self.phi.numpy(), -1))
                    fig_name = location + "/" + str(step) + ".pdf"
                    title_ = "$E_M(\phi)$ = " + str(compute_energy(self.phi)[0].numpy())
                    plt.figure()
                    plt.contourf(tf.transpose(self.phi))
                    plt.colorbar()
                    plt.title(title_)
                    plt.savefig(fig_name)
                    plt.close()
        
        if save_plots == True:
            file_E.close()
            file_A.close()
            

            E = np.genfromtxt(name_E, dtype='float32')
            A = np.genfromtxt(name_A, dtype='float32')
            plt.figure()
            plt.semilogy(E[:,0], linewidth=2)
            plt.grid(visible=True, which='both')
            plt.title("$E_M(\phi)$")
            plt.savefig(location + "/E.pdf")
            plt.close()

            plt.figure()
            plt.plot(A[:,0], linewidth=2)
            plt.grid(visible=True, which='both')
            plt.title("A($\phi$)")
            plt.savefig(location + "/A.pdf")
            plt.close()

In [ ]:
model = FONN(2)

location = "./Elastic_Bending_Results"

if not os.path.exists(location):
    os.makedirs(location)

In [ ]:
model.initial_training(1000)

In [ ]:
model.progressive_method(steps=501, K=10, save_plots=True, location=location)